<a href="https://colab.research.google.com/github/akalpit23/Explainable-techniques/blob/main/Explainable_Techniques_Assignment_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AIPI 590 - XAI | Assignment #05



## Akalpit Dawkhar

### **Explainable Techniques**

### Overview

In [2]:
# Please use this to connect your GitHub repository to your Google Colab notebook
# Connects to any needed files from GitHub and Google Drive
import os

# Remove Colab default sample_data
!rm -r ./sample_data

# Clone GitHub files to colab workspace
repo_name = "Explainable-techniques" # Change to your repo name
git_path = 'https://github.com/akalpit23/Explainable-techniques.git' #Change to your path
!git clone "{git_path}"

# Install dependencies from requirements.txt file
!pip install -r "{os.path.join(repo_name,'requirements.txt')}" #Add if using requirements.txt

# Change working directory to location of notebook
notebook_dir = 'Explainable_Techniques_Assignment_05.ipynb'
path_to_notebook = os.path.join(repo_name,notebook_dir)
%cd "{path_to_notebook}"
%ls

rm: cannot remove './sample_data': No such file or directory
fatal: destination path 'Explainable-techniques' already exists and is not an empty directory.
[Errno 20] Not a directory: 'Explainable-techniques/Explainable_Techniques_Assignment_05.ipynb'
/content
Explainable-techniques/


In [ ]:
import pandas as pd
import numpy as np
import torch
import xgboost as xgb
import shap
import lime
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.transforms import Compose, ToTensor, Normalize
from lime import lime_image
from sklearn.model_selection import train_test_split
from anchor import anchor_tabular
from sklearn.inspection import PartialDependenceDisplay
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.datasets import cifar10
from transformers import BertTokenizer, BertForSequenceClassification, pipeline
from PIL import Image
from skimage.segmentation import mark_boundaries
from alibi.explainers import AnchorImage

## Progress bar
from tqdm.notebook import tqdm

In [ ]:
# Load pre-trained ResNet34 model
model = models.resnet34(pretrained=True)
model.eval()

In [ ]:
# 3. Define image preprocessing steps (resize, normalize)
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# Load and preprocess the image
image_path = "/content/Explainable-techniques/Germanshepherd.jpg"
image = Image.open(image_path).convert('RGB')
preprocess = Compose([
    ToTensor(),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = preprocess(image).unsqueeze(0)

# 5. Predict with the model
output = model(input_tensor)
_, pred_class = torch.max(output, 1)


In [ ]:
# 6. Define the LIME explainer
explainer = lime_image.LimeImageExplainer()

# 7. Function to predict the output of perturbed images for LIME
def predict_fn(images):
    batch = torch.stack([preprocess(Image.fromarray(img)) for img in images])
    output = model(batch)
    return output.detach().numpy()

In [ ]:
# 8. Generate explanations for the prediction
explanation = explainer.explain_instance(np.array(image), predict_fn, top_labels=1, hide_color=0, num_samples=10)

In [ ]:
# 9. Visualize the explanation

# Get the top predicted label from the explanation
top_label = explanation.top_labels[0]

# First visualization: Show explanation with positive_only=False, num_features=10, hide_rest=False
temp, mask = explanation.get_image_and_mask(top_label, positive_only=False, num_features=10, hide_rest=False)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))  # Visualize the mask on the image
plt.axis('off')
plt.show()

# Second visualization: Show explanation with positive_only=True, num_features=5, hide_rest=True
temp, mask = explanation.get_image_and_mask(top_label, positive_only=True, num_features=5, hide_rest=True)
plt.imshow(mark_boundaries(temp / 2 + 0.5, mask))  # Visualize the mask on the image
plt.axis('off')
plt.show()

# Original visualization: Show explanation with positive_only=True, num_features=5, hide_rest=False
temp, mask = explanation.get_image_and_mask(top_label, positive_only=True, num_features=5, hide_rest=False)
plt.imshow(mark_boundaries(temp / 255.0, mask))  # Visualize the mask on the image
plt.axis('off')
plt.show()

In [ ]:
from torchvision import transforms

# Define your transform
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to PyTorch Tensor data type
    # Add any other transformations you need
])

pil_image = Image.open(image_path).convert("RGB")  # Load and convert image to RGB
image_tensor = transform(pil_image).unsqueeze(0)

In [ ]:
def predict_anchor(images):
    batch = torch.stack([preprocess(Image.fromarray(img.astype(np.uint8))) for img in images])
    output = model(batch)
    return output.detach().numpy()  # Convert PyTorch tensor to numpy array

In [ ]:
# Initialize the AnchorImage explainer
segmentation_fn = 'slic'
kwargs = {'n_segments': 15, 'compactness': 20, 'sigma': .5}  # Adjust segmentation parameters as needed
explainer = AnchorImage(predict_anchor, image_shape=(224, 224, 3), segmentation_fn=segmentation_fn,
                        segmentation_kwargs=kwargs)

# Explain the prediction for the input image (provide the image as a numpy array)
np_image = np.array(pil_image)
explanation = explainer.explain(np_image, threshold=0.95)

# Visualize the anchor
plt.imshow(explanation.anchor)
plt.title("Anchor Visualization")
plt.axis('off')  # Turn off axis
plt.show()

# Visualize the superpixels
plt.imshow(explanation.segments)
plt.title("Superpixels/Segments Visualization")
plt.axis('off')  # Turn off axis
plt.show()

In [ ]:
import torch
from torchvision.models import resnet34
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
from torchvision.transforms import Compose, Normalize, ToTensor
from PIL import Image
import numpy as np

# Define target layers (usually the last convolutional layer)
target_layers = [model.layer4[-1]]



cam = GradCAM(model=model, target_layers=target_layers)


# Define the target (e.g., class index 281 for 'tabby cat' in ImageNet)
targets = [ClassifierOutputTarget(281)]

# Generate the CAM
grayscale_cam = cam(input_tensor=input_tensor, targets=targets)

# Visualize the result
rgb_img = np.array(image)
visualization = show_cam_on_image(rgb_img/255.0, grayscale_cam[0], use_rgb=True)

# Display or save the visualization
import matplotlib.pyplot as plt
plt.imshow(visualization)
plt.axis('off')
plt.show()
